# 基于 Excel 的 QA 资料

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os

In [61]:
from textlong import idea
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings
from textlong.knowledge import LocalFilesLoader

## 加载文档

<div class="alert alert-warning">
    LocalFilesLoader默认支持Markdown、QAExcel，并实现混合加载。
</div>

In [4]:
qa = LocalFilesLoader(["QA", "python"])
qa.load()

[Document(page_content='低层建筑 \n场景：大于300平方米的银行营业厅;问题：是否需要设置自动喷水系统？', metadata={'answer': '问题答案：需要设立自动喷水灭火系统。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。&#10;&#10;', 'source': 'QA/demo.xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='低层建筑 \n场景：大于300平方米的银行营业厅&#10;问题：&#10;（1）是不是人员密集场所？&#10;（2）依据哪条规范？', metadata={'answer': '问题（1）答案：是人员密集场所。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条明确了人员密集场所的定义，并要求这类场所应采取相应的防火措施。银行营业厅通常会有较多的客户和工作人员，因此属于人员密集场所。&#10;&#10;问题（2）答案：《建筑设计防火规范》（GB50016-2014）。&#10;&#10;相关规范解释：该规范是中国建筑设计和消防审核的重要标准之一，其中包含了关于建筑分类、防火分区、安全疏散、消防设施等方面的详细规定，是判断一个场所是否为人员密集场所的重要依据之一。', 'source': 'QA/demo.xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='低层建筑 \n场景：5层高的大学实验楼&#10;（主要是化工、物理等实验室，并且采用了分体式空调）&#10;问题：是否需要设置自动喷淋系统吗，还是局部设置气体灭火系统？？', metadata={'answer': '问题答案：需要设置自动喷淋系统。&#10;&#10;相关规范解释：《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001）规定，大学实验楼作为教育科研建筑，尤其是含有化工、物理等实验室，存在较高的火灾风险，应设置自动喷淋系统。此外，根据实际情况评估，可考虑在特

## 构建缓存

In [62]:
from langchain_zhipu import ZhipuAIEmbeddings
model = ZhipuAIEmbeddings()
qa.cache_embeddings(model)

No embeddings to cached!


False

## 加载文本嵌入缓存

In [64]:
# load_embeddings 方法可以直接返回 from_embeddings 所需参数
emb, _m, metadata = qa.load_embeddings()

## 基于文本嵌入缓存查询

In [69]:
from langchain_community.vectorstores import FAISS
from textlong.knowledge import collect_docs

# 一行加载文本嵌入
db = FAISS.from_embeddings(*qa.load_embeddings(model))

# 微调向量查询策略
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1}
)

In [68]:
print((retriever|collect_docs).invoke('元组'))

Python是一种高级、解释型、交互式、面向对象的语言。关键语法包括：




- 数据类型：整数、浮点数、字符串、列表、元组、字典、集合。
- 控制流：if-elif-else条件语句，for和while循环。
- 函数定义：使用def关键字，支持默认参数、任意数量参数。
- 类与对象：class定义类，支持继承和多态。
- 异常处理：try-except块捕获和处理异常。
- 模块导入：使用import导入模块，from...import...来导入特定部分。
- 标准库：丰富的内置函数和模块，如sys、os、math、json等。
- 运算符：算术、比较、逻辑运算符，以及成员和身份测试。
- 文件操作：open函数用于文件读写，支持with语句管理资源。


> > > > > > >>>>>>>>>>>>>> 输出结果：
> > > > > > Python关键语法包括数据类型、控制流、函数定义、类与对象、异常处理、模块导入、标准库、运算符和文件操作等，具有简洁明了的特点，适合初学者和专业人士。


# RAG 查询

In [71]:
from textlong.memory import MemoryManager, WithMemoryBinding
from textlong.knowledge import LocalFilesLoader, collect_docs
from textlong.chain import create_qa_chain
from langchain_community.vectorstores import FAISS
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings

qa = LocalFilesLoader(["QA", "python"])
qa.get_files()

['QA/demo.xlsx', 'python/pandas语法摘要.md', 'python/python语法摘要.md']

In [72]:
es = qa.load_embeddings(ZhipuAIEmbeddings())
db = FAISS.from_embeddings(*es)

**不使用记忆：**

In [73]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1}
)
chain = create_qa_chain(ChatZhipuAI(), retriever)
for x in chain.stream("元组是什么？"):
    print(x.content, end="")

问题答案：元组是一种不可变序列。

相关规范解释：在Python中，元组是一个可以包含多个元素的有序集合，这些元素可以是不同的数据类型。元组用小括号()来表示，元素之间用逗号,分隔。由于不可变，一旦创建，元组内的元素不能被增加、删除或更改。这使得元组在确保数据不会被修改的情况下非常有用。

**使用默认对话ID(default)：**

In [74]:
memory = MemoryManager()
chain_with_memory = create_qa_chain(ChatZhipuAI(), db.as_retriever(), memory)
for x in chain_with_memory.stream({"input": "银行大厅要不要设置喷淋？"}):
    print(x.content, end="")

问题答案：需要设立自动喷水灭火系统。

相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。

**指定对话ID(1)：**

In [75]:
config = {"configurable": {"session_id": "1"}}
for x in chain_with_memory.stream({"input": "宿舍楼要不要设置喷淋？"}, config):
    print(x.content, end="")

问题答案：需要。

相关规范解释：《建筑设计防火规范》（GB 50016-2014）对宿舍楼等居住建筑也有相应的规定，要求在一定的条件下必须设置自动喷水灭火系统，尤其是对于多层或高层宿舍楼，以确保人员生命安全。

**检查历史对话：**

In [12]:
memory.get_memory("default")

[HumanMessage(content='银行大厅要不要设置喷淋？'),
 AIMessageChunk(content='问题答案：需要设置自动喷水灭火系统。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1719319881, 'index': 0, 'usage': {'prompt_tokens': 567, 'completion_tokens': 68, 'total_tokens': 635}}, id='8783880265663443368')]

In [13]:
memory.get_memory("1")

[HumanMessage(content='宿舍楼要不要设置喷淋？'),
 AIMessageChunk(content='问题答案：需要设置喷淋系统。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）对宿舍楼等居住建筑也有相应的要求，根据建筑的高度、人员密度等因素，可能需要配备自动喷水灭火系统，以保障人员安全。', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1719319888, 'index': 0, 'usage': {'prompt_tokens': 567, 'completion_tokens': 59, 'total_tokens': 626}}, id='8783891741816342315')]